# Dùng get-news để crawl dữ liệu tin tức

## 0. Change log
Ver0.1: Sử dụng thư viện [news-please](https://github.com/fhamborg/news-please), nhược điểm chưa lấy được URL của hình ảnh và videos

Ver0.2: Sử dụng [fake-useragent](https://pypi.org/project/fake-useragent/) để tạo _rotating proxy_ để có thể hỗ trợ lấy tin

Ver0.3: Sử dụng thêm thư viện BeautifulSoup để lấy thêm được URL của ảnh

## 1. Thư viện và các function

In [ ]:
import json, os
from newsplease import NewsPlease
# from bs4 import BeautifulSoup
# from fake_useragent import UserAgent

# from urllib.request import Request, urlopen

# ua = UserAgent()

### 1.1. Các function liên quan đến tin tức
Từ các url tin tức, chúng ta đưa chúng vào trong 1 list, hàm này dùng thư viên _NewsPlease_ để đưa dữ liệu tin tức lấy được vào 1 list

In [ ]:
def get_news_to_list(url_list):
    news_list = []
    for url in url_list:
        try:
            news_list.append(NewsPlease.from_url(url))
        except Exception as e: 
            print(e, ': ', url)
            continue
    return news_list

Đưa các dữ liệu sẵn có vào file JSON với cấu trúc là ```<json_prefix>_<json_no>.json```

Các thông số cho json.dump lần lượt là: __default = str__ đưa tất cả về string để tránh lỗi trong quá trình parse dữ liệu dạng _datetime_ sang json của các trường date\_modify, date\_download, date\_publish; __ensure\_ascii=False__ là để giữ các dữ liệu text ở dạng Unicode - để không bị lỗi khi đưa vào file json khi để text ở định dạng Unicode, thì trong phần __open()__ đặt thông số _encoding='utf-8'_. 

Các biến:
1. file\_name: tên file để lưu
2. file\_no: số thứ tự của file

In [ ]:
def dump_news_to_json(news_list, json_begin_no, json_prefix, file_path):
    file_no = json_begin_no
    for news in news_list:
        file_name = json_prefix + str(file_no) + '.json' 
        file_no += 1
        with open(os.path.join(file_path, file_name), 'w', encoding='utf-8') as outfile:
            json.dump(news.__dict__, outfile, indent=4, sort_keys=True, default=str, ensure_ascii=False)

### 1.2. Các function liên quan đến việc tạo rotating proxy
Tham khảo tại một số trang sau:
1. [Create a rotating proxy crawler in Python 3](https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/)

In [ ]:
def random_proxy(proxies):
  return random.randint(0, len(proxies) - 1)

In [ ]:
def retrive_proxies():
    proxies_req = Request('https://sslproxies.org/')
    proxies_req.add_header('User-Agent', ua.random)
    proxies_doc = urlopen(proxies_req).read().decode('utf8')
    
    

## 2. Các dữ liệu tin tức giả
URL về Fake News, tập hợp từ các bài viết có nội dung không được xác minh về nguồn tin, hoặc được xác nhận sai,... Có thể lưu url các bài báo trong 1 file text (mỗi url là 1 dòng)

In [ ]:
fake_news_urls = [
    "https://saostar.vn/the-gioi/ten-lua-phong-khong-ukraine-tu-ban-trung-toa-chung-cu-o-kiev-202202271128525058.html",
    "https://infonet.vietnamnet.vn/the-gioi/ukraine-ten-lua-ban-vao-nha-dan-khong-ben-nao-nhan-trach-nhiem-168057.html",
    "https://tuoitre.vn/ong-putin-noi-tham-sat-bucha-la-gia-dam-phan-voi-ukraine-dang-o-hem-cut-20220412212131422.htm",
    "https://baomoi.com/nga-noi-phuong-tay-giup-ukraine-nguy-tao-cao-buoc-tham-sat/c/42284586.epi",
    
]  

Bắt đầu crawl dữ liệu tin tức giả đưa vào list fake_articles

In [ ]:
fake_articles = get_news_to_list(fake_news_urls)

Tên của các file json sẽ chứa nội dung vừa crawl

In [ ]:
fake_no = 52
fake_prefix = 'VFND_Ac_Pro_Russia_'

Đưa dữ liệu đã lấy vào các file JSON để lưu trữ

In [ ]:
dump_news_to_json(fake_articles, fake_no, fake_prefix, './')

Test các kết quả trả ra khi crawl ở cell dưới

In [ ]:
i = 0
text1 = ""

for arc in fake_articles:
    num = fake_no + i
    text1 = text1 + fake_prefix + str(num) + ": [" + arc.title + "](" + arc.url + ")\n\n"

    i = i+1

print(text1)

## 3. Các dữ liệu tin tức thật

In [ ]:
real_news_urls = [
    "https://nld.com.vn/thoi-su-quoc-te/ukraine-phat-hien-may-chu-co-kha-nang-ven-man-vu-tham-sat-bucha-20220412131157701.htm",
    "https://trithucvn.org/the-gioi/tham-sat-o-bucha-ukraine-phat-hien-phong-tra-tan-cua-nga-trong-khu-vuc.html"

                 ]

Bắt đầu crawl dữ liệu, sử dụng thư viện news-please, các tin tức sẽ được lấy theo URL và được chứa trong mảng _real_\__article_.

In [ ]:
real_articles = get_news_to_list(real_news_urls)

Do thư viện _news-please_ không hỗ trợ để lấy hình ảnh vì thế chúng ta sẽ lấy hình ảnh thông qua thư viện _BeautifulSoup_. Sau mỗi lần lấy chúng ta tạo ra 1 đối tượng dictionary cho mỗi URL

In [ ]:
real_imgs = []
for real_url in real_news_urls:
    

Merge 2 đối tượng dictionary. Tham khảo từ: [How to merge two dictionaries in a single expression? - StackOverflow](https://stackoverflow.com/questions/38987/how-to-merge-two-dictionaries-in-a-single-expression)

Tương tự với việc crawl dữ liệu về tin tức giả. Ta có danh sách 1 số biến:
1. real\_name: tên file để lưu
2. real\_no: số thứ tự của file

In [ ]:
real_no = 6
real_prefix = 'VFND_Ac_Ukrainian_POV_'

In [ ]:
for real in real_articles:
    real_name = real_prefix + str(real_no) + '.json'
    real_no = real_no + 1
    with open(real_name, 'w', encoding='utf-8') as outfile:
        json.dump(real.__dict__, outfile, indent=4, sort_keys=True, default=str, ensure_ascii=False)

Test các kết quả trả ra khi crawl ở cell dưới

In [ ]:
j = 0
text2 = ""

for arc in real_articles:
    num = real_no + j
    text2 = text2 + real_prefix + str(num) + ": [" + arc.title + "](" + arc.url + ")\n\n"

    j = j+1

print(text2)
